**Question 4.1**

In [ ]:
from google.colab import files

uploaded = files.upload()


Saving orders_v_2022.csv to orders_v_2022.csv
Saving users_v (1).csv to users_v (1).csv


In [ ]:
!pip install apache-beam[gcp]


In [ ]:
!pip install apache-beam


In [ ]:
from types import NoneType
import apache_beam as beam


def parse_users(line):
    fields = line.split(',')
    if len(fields) >= 6:
        return fields[0], {'name': fields[1], 'gender': fields[2], 'age': fields[3], 'address': fields[4], 'date_joined': fields[5]}
    else:
        return NoneType

def parse_orders(line):
    fields = line.split(',')
    if len(fields) >= 4:
        return fields[0], {'order_id': fields[1], 'product': fields[2], 'amount': fields[3]}
    else:
        return None


with beam.Pipeline() as pipeline:

    users = (
        pipeline
        | 'Read Users' >> beam.io.ReadFromText('users_v (1).csv', skip_header_lines=1)
        | 'Parse Users' >> beam.Map(parse_users)
        | 'Filter Users' >> beam.Filter(lambda x: x is not None)
    )

    orders = (
        pipeline
        | 'Read Orders' >> beam.io.ReadFromText('orders_v_2022.csv', skip_header_lines=1)
        | 'Parse Orders' >> beam.Map(parse_orders)
        | 'Filter Orders' >> beam.Filter(lambda x: x is not None)  #
    )


    joined_data = (
        {'users': users, 'orders': orders}
        | 'Join Data' >> beam.CoGroupByKey()
    )

    joined_data | 'Print Output' >> beam.Map(print)


Streaming output truncated to the last 5000 lines.
('1562389;830;Broccoli', {'users': [], 'orders': [{'order_id': ' Artichoke', 'product': ' Turnip', 'amount': ' Beans;2021-03-30'}]})
('1562397;1334;Celery', {'users': [], 'orders': [{'order_id': ' Gherkin', 'product': ' Chinese Broccoli', 'amount': ' Bean Sprouts'}]})
('1562408;1964;Elephant Garlic', {'users': [], 'orders': [{'order_id': ' Asparagus', 'product': ' Okra', 'amount': ' Beet Greens;2021-03-31'}]})
('1562409;929;Plantain', {'users': [], 'orders': [{'order_id': ' Artichoke', 'product': ' Jerusalem Artichoke', 'amount': ' Burdock Root;2021-03-31'}]})
('1562454;608;Celtuce', {'users': [], 'orders': [{'order_id': ' Napa Cabbage', 'product': ' Okra', 'amount': ' Parsley Root;2021-04-01'}]})
('1562467;1515;Lotus Root', {'users': [], 'orders': [{'order_id': ' Chayote', 'product': ' Collard Greens', 'amount': ' Lemongrass;2021-04-01'}]})
('1562468;1069;Water Spinach', {'users': [], 'orders': [{'order_id': ' Broccoli', 'product': ' 

**Question 4.2**

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving orders_v_2022.csv to orders_v_2022 (1).csv
Saving users_v.csv to users_v.csv


In [ ]:
!mkdir -p data
!wget https://storage.googleapis.com/bdt-beam/users_v.csv -O data/users.csv
!wget https://storage.googleapis.com/bdt-beam/orders_v_2022.csv -O data/orders.csv


--2024-10-01 21:50:49--  https://storage.googleapis.com/bdt-beam/users_v.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.207, 108.177.11.207, 108.177.12.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143675 (140K) [text/csv]
Saving to: ‘data/users.csv’

data/users.csv      100%[===================>] 140.31K   534KB/s    in 0.3s    

2024-10-01 21:50:50 (534 KB/s) - ‘data/users.csv’ saved [143675/143675]

--2024-10-01 21:50:50--  https://storage.googleapis.com/bdt-beam/orders_v_2022.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.207, 108.177.11.207, 108.177.12.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73312907 (70M) [text/csv]
Saving to: ‘data/orders.csv’

data/orders.csv     100%[===================

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

pipeline_options = PipelineOptions()

def parse_users(line):
    fields = line.split(',')
    if len(fields) >= 3:
        user_id = fields[0]
        gender = fields[2]
        return (user_id, gender)
    return None

def parse_orders(line):
    fields = line.split(',')
    if len(fields) >= 2:
        user_id = fields[1]
        return (user_id, 1)
    return None

def calculate_average(gender, order_counts):
    total_orders = sum(order_counts)
    num_customers = len(order_counts)
    avg_orders = total_orders / num_customers if num_customers > 0 else 0
    return (gender, total_orders, num_customers, avg_orders)

with beam.Pipeline(options=pipeline_options) as p:
    users = (
        p | 'Read Users' >> beam.io.ReadFromText('data/users.csv', skip_header_lines=1)
          | 'Parse Users' >> beam.Map(parse_users)
          | 'Filter Invalid Users' >> beam.Filter(lambda x: x is not None)
    )

    orders = (
        p | 'Read Orders' >> beam.io.ReadFromText('data/orders.csv', skip_header_lines=1)
          | 'Parse Orders' >> beam.Map(parse_orders)
          | 'Filter Invalid Orders' >> beam.Filter(lambda x: x is not None)
    )

    joined_data = (
        {'users': users, 'orders': orders}
        | 'Join Users and Orders' >> beam.CoGroupByKey()
    )

    filtered_data = (
        joined_data | 'Filter Users with Orders' >> beam.Filter(lambda x: len(x[1]['users']) > 0 and len(x[1]['orders']) > 0)
    )


    gender_orders = (
        filtered_data | 'Map Gender to Orders' >> beam.Map(lambda x: (x[1]['users'][0], len(x[1]['orders'])))
    )


    grouped_by_gender = (
        gender_orders | 'Group by Gender' >> beam.GroupByKey()
    )

    avg_orders_by_gender = (
        grouped_by_gender | 'Calculate Average Orders' >> beam.Map(lambda x: calculate_average(x[0], x[1]))
    )


    avg_orders_by_gender | 'Print Average Orders Results' >> beam.Map(print)



**Question 4.3**

In [ ]:
import pandas as pd

users_df = pd.read_csv('users_v.csv')

orders_df = pd.read_csv('orders_v_2022.csv', delimiter=';')


users_df = users_df[['user_id', 'gender']]

orders_df = orders_df[['user_id']]

orders_count_df = orders_df.groupby('user_id').size().reset_index(name='order_count')

merged_df = pd.merge(users_df, orders_count_df, on='user_id', how='inner')

result = merged_df.groupby('gender')['order_count'].agg(
    total_orders='sum',
    num_customers='size',
    avg_orders='mean'
).reset_index()

print(result)


   gender  total_orders  num_customers  avg_orders
0  female        780295           1150  678.517391
1    male        818614           1207  678.222038


**Question 4.4**

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

pipeline_options = PipelineOptions()

def parse_users(line):
    fields = line.split(',')
    if len(fields) >= 3:
        user_id = fields[0]
        gender = fields[2]
        return (user_id, gender)
    return None

def parse_orders(line):
    fields = line.split(',')
    if len(fields) >= 2:
        user_id = fields[1]
        return (user_id, 1)
    return None

def calculate_average(gender, order_counts):
    total_orders = sum(order_counts)
    num_customers = len(order_counts)
    avg_orders = total_orders / num_customers if num_customers > 0 else 0
    return (gender, total_orders, num_customers, avg_orders)

with beam.Pipeline(options=pipeline_options) as p:

    users = (
        p | 'Read Users' >> beam.io.ReadFromText('users_v.csv', skip_header_lines=1)
          | 'Parse Users' >> beam.Map(parse_users)
          | 'Filter Invalid Users' >> beam.Filter(lambda x: x is not None)
    )

    orders = (
        p | 'Read Orders' >> beam.io.ReadFromText('orders_v_2022.csv', skip_header_lines=1)
          | 'Parse Orders' >> beam.Map(parse_orders)
          | 'Filter Invalid Orders' >> beam.Filter(lambda x: x is not None)
    )

    joined_data = (
        {'users': users, 'orders': orders}
        | 'Join Users and Orders' >> beam.CoGroupByKey()
    )

    filtered_data = (
        joined_data | 'Filter Users with Orders' >> beam.Filter(lambda x: len(x[1]['users']) > 0 and len(x[1]['orders']) > 0)
    )

    gender_orders = (
        filtered_data | 'Map Gender to Orders' >> beam.Map(lambda x: (x[1]['users'][0], len(x[1]['orders'])))
    )
    grouped_by_gender = (
        gender_orders | 'Group by Gender' >> beam.GroupByKey()
    )

    avg_orders_by_gender = (
        grouped_by_gender | 'Calculate Average Orders' >> beam.Map(lambda x: calculate_average(x[0], x[1]))
    )


    total_orders_processed = (
        orders | 'Count Total Orders' >> beam.combiners.Count.Globally()
    )

    avg_orders_by_gender | 'Print Average Orders Results' >> beam.Map(print)

    total_orders_processed | 'Print Total Orders Processed' >> beam.Map(lambda count: print(f"Total Orders Processed: {count}"))


Total Orders Processed: 1185020


**Question 4.5**

In [ ]:
pip install graphviz


In [ ]:
from graphviz import Digraph

# Create a directed graph
pipeline_graph = Digraph(comment='Beam Pipeline')

# Define the pipeline steps
pipeline_graph.node('A', 'Read Users')
pipeline_graph.node('B', 'Parse Users')
pipeline_graph.node('C', 'Read Orders')
pipeline_graph.node('D', 'Parse Orders')
pipeline_graph.node('E', 'Join Users and Orders')
pipeline_graph.node('F', 'Filter Valid Data')
pipeline_graph.node('G', 'Group by Gender')
pipeline_graph.node('H', 'Calculate Average Orders')
pipeline_graph.node('I', 'Count Total Orders')
pipeline_graph.node('J', 'Output Results')

# Connect the steps based on pipeline flow
pipeline_graph.edges(['AB', 'CD', 'EF', 'FG', 'GH', 'IJ'])
pipeline_graph.edge('C', 'E')
pipeline_graph.edge('B', 'E')
pipeline_graph.edge('H', 'J')
pipeline_graph.edge('I', 'J')

# Render the graph
pipeline_graph.render('beam_pipeline_graph', view=True, format='png')


'beam_pipeline_graph.png'

**Question 4.6**

In [3]:
!pip install apache-beam


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 19.7 MB/s eta 0:00:00


In [4]:
import apache_beam as beam


In [29]:
import apache_beam as beam

def parse_users(line):
    fields = line.split(',')
    if len(fields) >= 6:
        user_id = fields[0]
        gender = fields[2]
        age = int(fields[3])
        return (user_id, {'gender': gender, 'age': age})
    return None

def parse_orders(line):
    fields = line.split(';')
    if len(fields) >= 2:
        user_id = fields[1]
        return (user_id, 1)
    return None

def map_age_group(user_info):
    age = user_info['age']
    if 18 <= age < 28:
        return '18-27'
    elif 28 <= age < 38:
        return '28-37'
    elif 38 <= age < 46:
        return '38-45'
    elif 46 <= age < 55:
        return '46-54'
    return None

def map_gender_age_orders(data):
    user_info = data[1]['users'][0]
    orders = data[1]['orders']
    age_group = map_age_group(user_info)
    if age_group:
        return ((user_info['gender'], age_group), len(orders))
    return None

with beam.Pipeline() as p:
    users = (
        p | 'Read Users' >> beam.io.ReadFromText('users_v.csv', skip_header_lines=1)
          | 'Parse Users' >> beam.Map(parse_users)
          | 'Filter Invalid Users' >> beam.Filter(lambda x: x is not None)
    )
    orders = (
        p | 'Read Orders' >> beam.io.ReadFromText('orders_v_2022.csv', skip_header_lines=1)
          | 'Parse Orders' >> beam.Map(parse_orders)
          | 'Filter Invalid Orders' >> beam.Filter(lambda x: x is not None)
    )
    joined_data = (
        {'users': users, 'orders': orders}
        | 'Join Users and Orders' >> beam.CoGroupByKey()
    )
    gender_age_orders = (
        joined_data
        | 'Map Gender and Age Groups to Orders' >> beam.Map(map_gender_age_orders)
        | 'Filter Valid Age Groups' >> beam.Filter(lambda x: x is not None)
    )
    gender_age_orders | 'Print Intermediate Results' >> beam.Map(print)
    gender_age_orders = (
        gender_age_orders
        | 'Flatten Keys' >> beam.Map(lambda x: (x[0], x[1]))
    )

    total_orders_by_gender_age = (
        gender_age_orders
        | 'Group by Gender and Age Group' >> beam.CombinePerKey(sum)
    )
    total_orders_by_gender_age | 'Print Results' >> beam.Map(print)


(('female', '28-37'), 675)
(('male', '18-27'), 743)
(('male', '46-54'), 640)
(('female', '38-45'), 614)
(('male', '18-27'), 702)
(('male', '28-37'), 667)
(('female', '46-54'), 692)
(('male', '28-37'), 669)
(('male', '28-37'), 667)
(('female', '38-45'), 646)
(('female', '38-45'), 676)
(('female', '18-27'), 660)
(('female', '18-27'), 687)
(('male', '46-54'), 664)
(('female', '38-45'), 658)
(('male', '28-37'), 671)
(('female', '28-37'), 673)
(('male', '28-37'), 661)
(('female', '46-54'), 659)
(('male', '38-45'), 717)
(('female', '28-37'), 650)
(('female', '28-37'), 645)
(('male', '18-27'), 698)
(('female', '28-37'), 691)
(('male', '28-37'), 684)
(('male', '46-54'), 673)
(('male', '28-37'), 662)
(('female', '46-54'), 665)
(('male', '28-37'), 678)
(('female', '38-45'), 685)
(('female', '38-45'), 711)
(('male', '28-37'), 680)
(('female', '38-45'), 686)
(('male', '38-45'), 722)
(('female', '38-45'), 638)
(('female', '18-27'), 722)
(('male', '46-54'), 687)
(('female', '46-54'), 655)
(('male', 

In [32]:
import apache_beam as beam

def parse_users(line):
    fields = line.split(',')
    if len(fields) >= 6:
        user_id = fields[0]
        gender = fields[2]
        age = int(fields[3])
        return (user_id, {'gender': gender, 'age': age})
    return None

def parse_orders(line):
    fields = line.split(';')
    if len(fields) >= 4:
        user_id = fields[1]
        product_list = fields[2].lower()
        if 'yams' in product_list:
            return (user_id, 1)
    return None

def map_age_group(user_info):
    age = user_info['age']
    if 18 <= age < 28:
        return '18-27'
    elif 28 <= age < 38:
        return '28-37'
    return None

def map_gender_age_orders(data):
    user_info = data[1]['users'][0]
    orders = data[1]['orders']
    age_group = map_age_group(user_info)
    if age_group:
        return ((user_info['gender'], age_group), len(orders))
    return None

with beam.Pipeline() as p:
    users = (
        p | 'Read Users' >> beam.io.ReadFromText('users_v.csv', skip_header_lines=1)
          | 'Parse Users' >> beam.Map(parse_users)
          | 'Filter Invalid Users' >> beam.Filter(lambda x: x is not None)
    )

    orders = (
        p | 'Read Orders' >> beam.io.ReadFromText('orders_v_2022.csv', skip_header_lines=1)
          | 'Parse Orders' >> beam.Map(parse_orders)
          | 'Filter Valid Orders' >> beam.Filter(lambda x: x is not None)
    )

    joined_data = (
        {'users': users, 'orders': orders}
        | 'Join Users and Orders' >> beam.CoGroupByKey()
    )

    gender_age_orders = (
        joined_data
        | 'Map Gender and Age Groups to Yam Purchases' >> beam.Map(map_gender_age_orders)
        | 'Filter Valid Age Groups' >> beam.Filter(lambda x: x is not None)
        | 'Flatten Keys' >> beam.Map(lambda x: (x[0], x[1]))
    )

    gender_age_orders | 'Print Intermediate Results' >> beam.Map(print)

    total_yams_by_gender_age = (
        gender_age_orders
        | 'Group by Gender and Age Group' >> beam.CombinePerKey(sum)
    )

    total_yams_by_gender_age | 'Print Results' >> beam.Map(print)


(('male', '28-37'), 10)
(('male', '28-37'), 20)
(('male', '28-37'), 13)
(('male', '28-37'), 14)
(('male', '18-27'), 17)
(('male', '28-37'), 13)
(('male', '28-37'), 13)
(('female', '18-27'), 14)
(('male', '18-27'), 9)
(('male', '18-27'), 18)
(('female', '28-37'), 19)
(('female', '18-27'), 24)
(('male', '18-27'), 12)
(('female', '28-37'), 14)
(('female', '18-27'), 12)
(('male', '18-27'), 9)
(('male', '18-27'), 16)
(('female', '28-37'), 18)
(('male', '18-27'), 19)
(('male', '28-37'), 14)
(('female', '28-37'), 13)
(('female', '18-27'), 15)
(('male', '18-27'), 15)
(('female', '28-37'), 18)
(('male', '28-37'), 18)
(('female', '28-37'), 13)
(('male', '18-27'), 13)
(('female', '28-37'), 14)
(('male', '28-37'), 10)
(('female', '18-27'), 14)
(('male', '28-37'), 14)
(('female', '28-37'), 12)
(('male', '28-37'), 17)
(('female', '18-27'), 17)
(('female', '18-27'), 14)
(('male', '18-27'), 22)
(('male', '28-37'), 15)
(('female', '28-37'), 9)
(('male', '18-27'), 17)
(('female', '18-27'), 13)
(('female